The purpose is to use Dropout in VAE in order to generate data.

Origin VAE uses latent values to generate the data.
However, with Dropout VAE, we can use some origin data to generate.


The dataset used in notebook: [Customer Support](https://www.ibm.com/communities/analytics/watson-analytics-blog/guide-to-sample-datasets/)

More example about VAE: [Modeling Telecom customer churn](https://towardsdatascience.com/modeling-telecom-customer-churn-with-variational-autoencoder-4e5cf6194871)

Origin code for VAE in keras: [Building Autoencoders in keras](https://blog.keras.io/building-autoencoders-in-keras.html)

# Import Libraries

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import random
import argparse
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from tensorflow import set_random_seed

from keras import regularizers
from keras import backend as K
from keras.models import Model
from keras.utils import plot_model
from keras.losses import mse, binary_crossentropy
from keras.layers import Lambda, Input, Dense, Dropout

set_random_seed(1)
np.random.seed(1)
random.seed(1)

Using TensorFlow backend.


# Path to dataset

In [0]:
path = '/Dataset/WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [0]:
intermediate_dim = 512

# Define VAE

## Read Data

In [6]:
import pandas as pd
na_values = {'?', ' '}
df = pd.read_csv(path,
                 sep=',',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
                 
df.fillna(method='ffill', inplace=True)
df.dropna(axis=1, how='any', inplace=True)
df.drop(['customerID'], axis=1, inplace=True)
df = df.reset_index(drop=True)
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null float64
Churn               7043 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB
None
   ge

## Change Categorical data --> numbers

In [7]:
colnums = len(df.columns)
for i in df.columns:
    try:
        if df[i].dtype.name == 'object':
            df[i] = df[i].astype('category')
        else:
            df[i].astype('float32')
    except:
        continue
print(df.head())

   gender  SeniorCitizen Partner  ... MonthlyCharges  TotalCharges Churn
0  Female              0     Yes  ...          29.85         29.85    No
1    Male              0      No  ...          56.95       1889.50    No
2    Male              0      No  ...          53.85        108.15   Yes
3    Male              0      No  ...          42.30       1840.75    No
4  Female              0      No  ...          70.70        151.65   Yes

[5 rows x 20 columns]


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
vals = df.values.copy()
total_nums = len(vals)

train, validation = train_test_split(df, test_size=0.5, 
                                     random_state=42, 
                                     shuffle=True)

validation = validation.reindex(sorted(validation.columns), axis=1)
validation.to_csv(path + '_For_Test.csv')
print(validation.head())

     Churn        Contract Dependents  ... TotalCharges  gender  tenure
185    Yes  Month-to-month         No  ...        24.80  Female       1
2715    No  Month-to-month         No  ...       996.45    Male      41
3825    No        Two year        Yes  ...      1031.70  Female      52
1807   Yes  Month-to-month         No  ...        76.35  Female       1
132     No        Two year         No  ...      3260.10    Male      67

[5 rows x 20 columns]


In [9]:
df = train
categorical = df.select_dtypes(['category']).columns
print(categorical)
for f in categorical:
    dummies = pd.get_dummies(df[f], prefix = f, prefix_sep = '_')
    df = pd.concat([df, dummies], axis = 1)
    
# drop original categorical features
df.drop(categorical, axis = 1, inplace = True)

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Churn'],
      dtype='object')


In [10]:
train = df.values.copy()
train.astype('float32')
scaler = MinMaxScaler()
d = {}
train = scaler.fit_transform(train)
x_train, x_test = train_test_split(train, test_size=0.5,
                                  random_state=24,
                                  shuffle=True)
print(x_train.shape)
print(x_test.shape)


(1760, 47)
(1761, 47)


In [11]:
original_dim = x_train.shape[1]
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape)
print(x_test.shape)

(1760, 47)
(1761, 47)


In [0]:
from keras.layers import BatchNormalization
class VAE:
    def __init__(self, input_shape=(original_dim,), 
                 intermediate_dim=128, latent_dim=2, summary=False):
        
        self._build_model(input_shape,
                         intermediate_dim, 
                          latent_dim, summary)
    
    def _build_model(self, input_shape, intermediate_dim, latent_dim,
                    summary=False):
        inputs = Input(shape=input_shape, name='encoder_input')
        x = inputs
        x = Dense(intermediate_dim, activation='relu')(x)
        x = Dense(intermediate_dim//2, activation='linear')(x)
        
        z_mean = Dense(latent_dim, name='z_mean')(x)
        z_log_var = Dense(latent_dim, name='z_log_var')(x)

        z = Lambda(self.sampling, output_shape=(latent_dim,), 
                   name='z')([z_mean, z_log_var])

        self.encoder = Model(inputs, [z_mean, z_log_var, z], 
                        name='encoder')
        
        latent_inputs = Input(shape=(latent_dim,), 
                              name='z_sampling')
        x = latent_inputs
        x = Dense(intermediate_dim//2, activation='relu')(x)
        x = Dense(intermediate_dim, activation='relu')(x)
        outputs = Dense(original_dim, activation='sigmoid')(x)

        self.decoder = Model(latent_inputs, outputs, name='decoder')
        outputs = self.decoder(self.encoder(inputs)[2])
        self.vae = Model(inputs, outputs, name='vae_mlp')
        
        reconstruction_loss = binary_crossentropy(inputs, outputs)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        
        vae_loss = K.mean(reconstruction_loss + kl_loss)	
        
        self.vae.add_loss(vae_loss)
        self.vae.compile(optimizer='adam')
        if summary: 
            print(self.vae.summary())
        
    def sampling(self, args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon
        
    def fit(self, x_train, x_test, epochs=100, batch_size=100,
           verbose=1):
        self.vae.fit(x_train, 
            shuffle=True,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(x_test, None))
    
    def encoder_predict(self, x_test, batch_size=100):
        return self.encoder.predict(x_test,
                                   batch_size=batch_size)
    
    def generate(self, latent_val, batch_size=100):
        return self.decoder.predict(latent_val)
    
    def predict(self, x_test, batch_size=1):
        prediction = self.vae.predict(x_test)
        return prediction

## Training VAE

Just let the last value to test

In [13]:
latent_dim = original_dim//2
if latent_dim < 2:
    latent_dim = 2
vae = VAE(intermediate_dim=intermediate_dim, latent_dim=latent_dim)
vae.fit(x_train, x_test, epochs=150)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 1760 samples, validate on 1761 samples
Epoch 1/150
1760/1760 [==============================] - 1s 756us/step - loss: 29.4586 - val_loss: 28.3182
Epoch 2/150
1760/1760 [==============================] - 0s 99us/step - loss: 27.1679 - val_loss: 25.4552
Epoch 3/150
1760/1760 [==============================] - 0s 95us/step - loss: 23.8962 - val_loss: 23.0662
Epoch 4/150
1760/1760 [==============================] - 0s 93us/step - loss: 22.6596 - val_loss: 22.3829
Epoch 5/150
1760/1760 [==============================] - 0s 92us/step - loss: 22.1373 - val_loss: 21.9027
Epoch 6/150
1760/1760 [==============================] - 0s 92us/step - loss: 21.6997 - val_loss: 21.6288
Epoch 7/150
1760/1760 [==============================] - 0s 90us/step - loss: 21.3673 - val_loss: 20.9163
Epoch 8/150
1760/1760 [==============================] - 0s 93us/step - loss: 20.9183 - v

In [14]:
np.random.seed(1)
x_test = np.reshape(x_test, (-1, original_dim))
x_test_encoded = vae.encoder.predict(x_test)
x_test_encoded = np.asarray(x_test_encoded)

print(x_test_encoded.shape)

(3, 1761, 23)


In [15]:
total_nums = 2
results = []
for i in range(x_test_encoded.shape[1]):
    latent_gen = []
    for j in range(x_test_encoded.shape[2]):
        mean_v1 = x_test_encoded[0, i, j]
        log_var_v1 = x_test_encoded[1, i, j]

        v1 = [mean_v1 + np.exp(log_var_v1/2)*np.random.normal(0., 1., 1) for _ in range(total_nums)]
        v1 = np.asarray(v1).flatten()
        latent_gen.append(v1)
    
    input_val = np.asarray(latent_gen)
    input_val = input_val.reshape((-1, latent_dim))
    results.append(vae.generate(input_val))
        
results = np.asarray(results)
results = np.reshape(results, (-1, original_dim))
print(results.shape)
results = scaler.inverse_transform(results)

(3522, 47)


In [16]:
print(len(results[:, 1]))
print(results[0, 0])

3522
0.11541468


In [0]:
d = {}
names = list(df)
for i, name in enumerate(names):
    d[name] = results[:, i]
df = pd.DataFrame(data=d)

In [18]:
names = list(df)
c_dict = {}
for n in names:
    if '_' in n:
        index = n.index('_')
        c_dict[n[:index]] = [c for c in names if n[:index+1] in c]
values = []
for key, items in c_dict.items():
    dummies = df[items]
    d_names = list(dummies)
    c_dict = {}
    for n in d_names:
        c_dict[n] = n[n.index('_')+1:]
    dummies.rename(columns=c_dict, 
                   inplace=True)
    df[key] = dummies.idxmax(axis=1)
    df.drop(items, axis=1, inplace=True)
print(df.head())

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


   SeniorCitizen     tenure  ...            PaymentMethod  Churn
0       0.115415  31.163046  ...  Credit card (automatic)     No
1       0.002976   9.138953  ...             Mailed check     No
2       0.176751  10.862296  ...         Electronic check    Yes
3       0.323760  27.991091  ...         Electronic check    Yes
4       0.023901  38.340378  ...  Credit card (automatic)     No

[5 rows x 20 columns]


In [0]:
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv(path + '_vae.csv')

# Dropout VAE

## Read Data

In [20]:
import pandas as pd
na_values = {'?', ' '}
df = pd.read_csv(path,
                 sep=',',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
                 
df.fillna(method='ffill', inplace=True)
df.dropna(axis=1, how='any', inplace=True)
df.drop(['customerID'], axis=1, inplace=True)
df = df.reset_index(drop=True)
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null float64
Churn               7043 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB
None
   ge

## Change Categorical data --> numbers

In [21]:
colnums = len(df.columns)
for i in df.columns:
    try:
        if df[i].dtype.name == 'object':
            df[i] = df[i].astype('category')
        else:
            df[i].astype('float32')
    except:
        continue
print(df.head())

   gender  SeniorCitizen Partner  ... MonthlyCharges  TotalCharges Churn
0  Female              0     Yes  ...          29.85         29.85    No
1    Male              0      No  ...          56.95       1889.50    No
2    Male              0      No  ...          53.85        108.15   Yes
3    Male              0      No  ...          42.30       1840.75    No
4  Female              0      No  ...          70.70        151.65   Yes

[5 rows x 20 columns]


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
vals = df.values.copy()
total_nums = len(vals)

train, validation = train_test_split(df, test_size=0.5, 
                                     random_state=42, 
                                     shuffle=True)

validation = validation.reindex(sorted(validation.columns), axis=1)
validation.to_csv(path + '_For_Test.csv')
print(validation.head())

     Churn        Contract Dependents  ... TotalCharges  gender  tenure
185    Yes  Month-to-month         No  ...        24.80  Female       1
2715    No  Month-to-month         No  ...       996.45    Male      41
3825    No        Two year        Yes  ...      1031.70  Female      52
1807   Yes  Month-to-month         No  ...        76.35  Female       1
132     No        Two year         No  ...      3260.10    Male      67

[5 rows x 20 columns]


In [23]:
df = train
categorical = df.select_dtypes(['category']).columns
print(categorical)
for f in categorical:
    dummies = pd.get_dummies(df[f], prefix = f, prefix_sep = '_')
    df = pd.concat([df, dummies], axis = 1)
    
# drop original categorical features
df.drop(categorical, axis = 1, inplace = True)

Index(['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Churn'],
      dtype='object')


In [0]:
train = df.values.copy()
train.astype('float32')
scaler = MinMaxScaler()
d = {}
train = scaler.fit_transform(train)
x_train, x_test = train_test_split(train, test_size=0.5,
                                  random_state=24,
                                  shuffle=True)

In [25]:
original_dim = x_train.shape[1]
x_train = np.reshape(x_train, [-1, original_dim])
x_test = np.reshape(x_test, [-1, original_dim])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
print(x_train.shape)
print(x_test.shape)

(1760, 47)
(1761, 47)


In [0]:
from keras.regularizers import l2
from keras.losses import categorical_crossentropy
class DropoutVAE:
    def __init__(self, input_shape=(original_dim,), 
                 intermediate_dim=32, latent_dim=3, dropout=0.05, 
                 summary=False):
        
        self._build_model(input_shape,
                         intermediate_dim, 
                          latent_dim, summary,
                          dropout)
    
    def _build_model(self, input_shape, intermediate_dim, latent_dim,
                    summary=False, dropout=0.05):
        inputs = Input(shape=input_shape, name='encoder_input')
        x = inputs
        x = Dense(intermediate_dim, activation='relu')(x)
        x = Dense(intermediate_dim//2, activation='relu')(x)
        
        z_mean = Dense(latent_dim, name='z_mean')(x)
        z_log_var = Dense(latent_dim, name='z_log_var')(x)

        z = Lambda(self.sampling, output_shape=(latent_dim,), 
                   name='z')([z_mean, z_log_var])

        self.encoder = Model(inputs, [z_mean, z_log_var, z], 
                        name='encoder')
        
        latent_inputs = Input(shape=(latent_dim,), 
                              name='z_sampling')
        x = latent_inputs
        x = Dense(intermediate_dim//2, activation='relu',
                 kernel_regularizer=l2(1e-4),
                 bias_regularizer=l2(1e-4))(x)
        x = Dropout(dropout)(x)
        x = Dense(intermediate_dim, activation='relu',
                 kernel_regularizer=l2(1e-4),
                 bias_regularizer=l2(1e-4))(x)
        x = Dropout(dropout)(x)
        outputs = Dense(original_dim, activation='sigmoid',
                       kernel_regularizer=l2(1e-4),
                       bias_regularizer=l2(1e-4))(x)

        self.decoder = Model(latent_inputs, 
                             outputs, 
                             name='decoder')
        outputs = self.decoder(self.encoder(inputs)[2])
        self.vae = Model(inputs, outputs, 
                         name='vae_mlp')
        
        reconstruction_loss = binary_crossentropy(inputs, outputs)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        
        vae_loss = K.mean(reconstruction_loss + kl_loss)	
        
        self.vae.add_loss(vae_loss)
        self.vae.compile(optimizer='adam')
        if summary: 
            print(self.vae.summary())
        
    def sampling(self, args):
        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon
        
    def fit(self, x_train, x_test, epochs=100, batch_size=100,
           verbose=1):
        self.vae.fit(x_train, 
            shuffle=True,
            epochs=epochs,
            batch_size=batch_size,
            verbose=verbose,
            validation_data=(x_test, None))
    
    def encoder_predict(self, x_test, batch_size=100):
        return self.encoder.predict(x_test,
                                   batch_size=batch_size)
    
    def generate(self, latent_val, batch_size=100):
        return self.decoder.predict(latent_val)
    
    def predict(self, x_test, batch_size=1, nums=1000):
        predict_stochastic = K.function([self.vae.layers[0].input,
                                        K.learning_phase()],
                                        [self.vae.get_output_at(0)])
        Yt_hat = np.array([self.vae.predict(x_test) for _ in range(nums)]) 
        return Yt_hat

## Train and evaluate Dropout VAE

In [27]:
latent_dim = original_dim//2
if latent_dim < 2:
    latent_dim = 2
vae = DropoutVAE(intermediate_dim=intermediate_dim,
                 dropout=0.2, latent_dim=latent_dim,
                 summary=True)
vae.fit(x_train, x_test, epochs=100)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   (None, 47)                0         
_________________________________________________________________
encoder (Model)              [(None, 23), (None, 23),  167726    
_________________________________________________________________
decoder (Model)              (None, 47)                161839    
Total params: 329,565
Trainable params: 329,565
Non-trainable params: 0
_________________________________________________________________
None
Train on 1760 samples, validate on 1761 samples
Epoch 1/100
1760/1760 [==============================] - 1s 508us/step - loss: 29.6260 - val_loss: 28.4184
Epoch 2/100
1760/1760 [==============================] - 0s 103us/step - loss: 28.0914 - val_loss: 27.1512
Epoch 3/100
1760/1

In [28]:
np.random.seed(1)
x_test = np.reshape(x_test, (-1, original_dim))
print(x_test.shape)
print(x_test[0].reshape(-1, original_dim).shape)

(1761, 47)
(1, 47)


In [0]:
total_nums = 2
results = []
for i in range(x_test.shape[0]):
    x_test_encoded = vae.predict(x_test[i].reshape(-1, original_dim), 
                                 nums=total_nums)
    x_test_encoded = x_test_encoded.reshape(total_nums, original_dim)
    results.append(x_test_encoded)
results = np.asarray(results)
results = results.reshape(total_nums*results.shape[0], original_dim)
results = scaler.inverse_transform(results)

## Results

In [0]:
d = {}
names = list(df)
for i, name in enumerate(names):
    d[name] = results[:, i]
df = pd.DataFrame(data=d)

In [31]:
names = list(df)
c_dict = {}
for n in names:
    if '_' in n:
        index = n.index('_')
        c_dict[n[:index]] = [c for c in names if n[:index+1] in c]
values = []
for key, items in c_dict.items():
    dummies = df[items]
    d_names = list(dummies)
    c_dict = {}
    for n in d_names:
        c_dict[n] = n[n.index('_')+1:]
    dummies.rename(columns=c_dict, 
                   inplace=True)
    df[key] = dummies.idxmax(axis=1)
    df.drop(items, axis=1, inplace=True)
print(df.head())

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


   SeniorCitizen     tenure  ...     PaymentMethod  Churn
0       0.003236   3.630666  ...      Mailed check     No
1       0.008458   6.142739  ...      Mailed check     No
2       0.548011  27.437634  ...  Electronic check     No
3       0.345497  35.393139  ...  Electronic check     No
4       0.016636  17.806929  ...      Mailed check     No

[5 rows x 20 columns]


In [0]:
df = df.reindex(sorted(df.columns), axis=1)
df.to_csv(path + '_dropout.csv')

# Transform categorical -> number

In [0]:

validation.to_csv(path + '_For_Test.csv')
df = pd.read_csv(path + '_For_Test.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df_mc = pd.read_csv(path + '_dropout.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df_vae = pd.read_csv(path + '_vae.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df.drop('Unnamed: 0', axis=1, inplace=True)
df_mc.drop('Unnamed: 0', axis=1, inplace=True)
df_vae.drop('Unnamed: 0', axis=1, inplace=True)
names = list(df)

In [34]:
from sklearn.preprocessing import LabelEncoder
colnums = len(df.columns)
for i in df.columns:
    try:
        if df[i].dtype.name == 'object':
            df[i] = df[i].astype('category')
    except:
        continue
cat_columns = df.select_dtypes(['category']).columns
print(cat_columns)
for col in cat_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].values)
    df_mc[col] = le.transform(df_mc[col].values)
    df_vae[col] = le.transform(df_vae[col].values)
    

Index(['Churn', 'Contract', 'Dependents', 'DeviceProtection',
       'InternetService', 'MultipleLines', 'OnlineBackup', 'OnlineSecurity',
       'PaperlessBilling', 'Partner', 'PaymentMethod', 'PhoneService',
       'StreamingMovies', 'StreamingTV', 'TechSupport', 'gender'],
      dtype='object')


In [0]:
df = df.reindex(sorted(df.columns), axis=1)
df_mc = df_mc.reindex(sorted(df_mc.columns), axis=1)
df_vae = df_vae.reindex(sorted(df_vae.columns), axis=1)
df.to_csv(path + '_For_Test_encoded.csv')
df_mc.to_csv(path + '_dropout_encoded.csv')
df_vae.to_csv(path + '_vae_encoded.csv')

# Predicting with generated data

In [0]:
df = pd.read_csv(path + '_For_Test_encoded.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df_mc = pd.read_csv(path + '_dropout_encoded.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df_vae = pd.read_csv(path + '_vae_encoded.csv',
                 na_filter=True, 
                 verbose=False, 
                 skip_blank_lines=True, 
                 na_values=na_values,
                 keep_default_na=False)
df.drop('Unnamed: 0', axis=1, inplace=True)
df_mc.drop('Unnamed: 0', axis=1, inplace=True)
df_vae.drop('Unnamed: 0', axis=1, inplace=True)

In [37]:
print(df.head())
print(df_mc.head())
print(df_vae.head())

   Churn  Contract  Dependents  ...  TotalCharges  gender  tenure
0      1         0           0  ...         24.80       0       1
1      0         0           0  ...        996.45       1      41
2      0         2           1  ...       1031.70       0      52
3      1         0           0  ...         76.35       0       1
4      0         2           0  ...       3260.10       1      67

[5 rows x 20 columns]
   Churn  Contract  Dependents  ...  TotalCharges  gender     tenure
0      0         0           0  ...      68.44079       0   3.630666
1      0         0           0  ...      71.27217       0   6.142739
2      0         0           0  ...    2544.45700       0  27.437634
3      0         0           0  ...    3410.82640       0  35.393140
4      0         1           0  ...     286.11273       0  17.806929

[5 rows x 20 columns]
   Churn  Contract  Dependents  ...  TotalCharges  gender     tenure
0      0         0           0  ...    1741.66280       0  31.163046
1     

In [38]:
y = df['Churn'].values
df.drop(['Churn'], axis=1, inplace=True)
X = df.values
print(y.shape)
print(X.shape)

(3522,)
(3522, 19)


In [39]:
y_mc = df_mc['Churn'].values
df_mc.drop(['Churn'], axis=1, inplace=True)
X_mc = df_mc.values
print(y_mc.shape)
print(X_mc.shape)

(3522,)
(3522, 19)


In [40]:
y_vae = df_vae['Churn'].values
df_vae.drop(['Churn'], axis=1, inplace=True)
X_vae = df_vae.values
print(y_vae.shape)
print(X_vae.shape)

(3522,)
(3522, 19)


## Cross validation

### Original data

In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, 
                             max_depth=2, random_state=42)
scores = cross_val_score(clf, X, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.03)


### DropoutVAE generated data with original Outcome

In [42]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, 
                             max_depth=2, random_state=42)
scores = cross_val_score(clf, X_mc, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.73 (+/- 0.00)


### VAE generated data with original outcome

In [43]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, 
                             max_depth=2, random_state=42)
scores = cross_val_score(clf, X_vae, y, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.73 (+/- 0.00)


### Original data with Dropout VAE outcome

In [44]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, 
                             max_depth=2, random_state=42)
scores = cross_val_score(clf, X, y_mc, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.77 (+/- 0.00)


### Original data with VAE outcome

In [45]:
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=100, 
                             max_depth=2, random_state=42)
scores = cross_val_score(clf, X, y_vae, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.73 (+/- 0.00)
